In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from PIL import Image
from scipy import sparse
from scipy.sparse import diags
from scipy.sparse.linalg import factorized

In [2]:
xi, xf, dx = 0, 2*np.pi, 1e-2
ti, tf, dt = 0, 6, 1e-2

x = np.arange(xi, xf+dx, dx)
t = np.arange(ti, tf+dt, dt)

Nt, Nx = len(t), len(x)

def u0(x):
    return np.exp(-(x-xf/2)**2)

def D_t():
    "4th Order centred finite difference with periodic boundary conditions"
    D = 1/(12*dx) * diags([1, -8, 0, 8, -1], [-2, -1, 0, 1, 2], shape=(Nx, Nx)).toarray()
    D[0, [-2, -1]] = [1*1/(12*dx), -8*1/(12*dx)]
    D[1, [-1]] = [1*1/(12*dx)]
    D[-1, [0, 1]] = [8*1/(12*dx), -1*1/(12*dx)]
    D[-2, [0]] = [-1*1/(12*dx)]
    return sparse.csr_matrix(D)

c = np.pi/3
u = np.zeros((Nt, Nx))
I = diags([1], [0], shape=(Nx, Nx))
A = (I - dt*c/2 * D_t())
B = (I + dt*c/2 * D_t())
solve_A = factorized(A.tocsc())

In [3]:
for j, tj in enumerate(t):
    if j == 0:
        u[0, :] = u0(x)
    else:
        uj = u[j-1, :]
        u[j, :] = solve_A(B @ uj)
    
    if np.max(np.abs(u[j, :])) > 1e2:
        print(f"Blow up at time {tj:.3f}s")
        break
    
    if j % int(1/dt * 1e-1) == 0:
        print(f"Time {tj:.3f}s, u_max = {np.max(np.abs(u[j, :])):.3f}")


Time 0.000s, u_max = 1.000
Time 0.100s, u_max = 1.000
Time 0.200s, u_max = 1.000
Time 0.300s, u_max = 1.000
Time 0.400s, u_max = 1.000
Time 0.500s, u_max = 1.000
Time 0.600s, u_max = 1.000
Time 0.700s, u_max = 1.000
Time 0.800s, u_max = 1.000
Time 0.900s, u_max = 1.000
Time 1.000s, u_max = 1.000
Time 1.100s, u_max = 1.000
Time 1.200s, u_max = 1.000
Time 1.300s, u_max = 1.000
Time 1.400s, u_max = 1.000
Time 1.500s, u_max = 1.000
Time 1.600s, u_max = 1.000
Time 1.700s, u_max = 1.000
Time 1.800s, u_max = 1.000
Time 1.900s, u_max = 1.000
Time 2.000s, u_max = 1.000
Time 2.100s, u_max = 1.000
Time 2.200s, u_max = 1.000
Time 2.300s, u_max = 1.000
Time 2.400s, u_max = 1.000
Time 2.500s, u_max = 1.000
Time 2.600s, u_max = 1.000
Time 2.700s, u_max = 1.000
Time 2.800s, u_max = 1.000
Time 2.900s, u_max = 1.000
Time 3.000s, u_max = 1.000
Time 3.100s, u_max = 1.000
Time 3.200s, u_max = 1.000
Time 3.300s, u_max = 1.000
Time 3.400s, u_max = 1.000
Time 3.500s, u_max = 1.000
Time 3.600s, u_max = 1.000
T

In [4]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

def make_plot(save=False, file_name=f"PDE_solution.png"):
    "Helper function for plotting"
    fig, ax = plt.subplots()
    ax.plot(x, u[i, :])
    ax.set_xlim(xi, xf)
    ax.set_ylim(-1.05, 1.05)
    ax.grid()
    ax.set_title(f"Time = {t[i]:.3f}s")
    if save:
        fig.savefig(file_name, bbox_inches="tight", pad_inches=0.1, dpi=100, facecolor="white")

In [5]:
import os
import shutil

folder_path="plots"
for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # Remove file or symbolic link
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove subdirectory and its contents
    except Exception as e:
        print(f"Failed to delete {item_path}. Reason: {e}")

In [6]:
files = []
fps   = 20
frame_indicies = np.linspace(0, Nt-1, int(fps*tf), dtype=int)
for i in frame_indicies:
    ti = t[i]
    file = f"plots/PDE_sol_{ti:.5f}.png"
    make_plot(save=True, file_name=file)
    files.append(file)
    plt.close("all")
    
save_gif_PIL("gifs/Advection_Solution.gif", files, fps=fps, loop=0)